In [14]:
import numpy as np
import numpy.random as rand

In [129]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")

while True: 
  try:
    num_mines = int(input("Enter number of mines: "))
    if num_mines > dimension**2:
      raise ValueError
    break
  except ValueError:
    print("Must be an integer and less than dimension^2")



Enter dimension of board: 20
Enter number of mines: 20


In [138]:
#initialize board
board = np.zeros((dimension, dimension))

remaining_spaces = set([])
for i in range(dimension):
  for j in range(dimension):
    remaining_spaces.add((i,j))

#initialize mines
counter = 0
mines_set = set([])
while counter < num_mines:
  x = rand.randint(0, dimension)
  y = rand.randint(0, dimension)
  if x == 0 and y == 0:
    continue
  if (x,y) in mines_set:
    continue
  else:
    mines_set.add((x,y))
    board[x][y] = 1
    counter += 1


print(board)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [68]:
#clue that our agent will receive
#returns the total sum of the neighbors of current_position
def clue(current_position, board,  dimension):
  x = current_position[0]
  y = current_position[1]
  sum = 0
  
  if x+1 < dimension:
    sum += board[x+1][y]
    if y+1 < dimension:
      sum += board[x+1][y+1]
    if y-1 >= 0:
      sum += board[x+1][y-1]

  if x-1 >= 0:
    sum += board[x-1][y]
    if y+1 < dimension:
      sum += board[x-1][y+1]
    if y-1 >= 0:
      sum += board[x-1][y-1]

  if y+1 < dimension:
    sum += board[x][y+1]
  
  if y-1 >= 0:
    sum += board[x][y-1]

  return sum
  

In [139]:
start = (0,0)
board_perception = np.zeros((dimension, dimension)) #-1: unknown, 0: clear, 1: mine
#clue_perception = np.zeros((dimension, dimension))
for i in range(dimension):
  for j in range(dimension):
    board_perception[i][j] = -1
    #clue_perception[i][j] = -1

knowledge_base = [] #(0,0): location, 0: value(0 or 1)
board_perception[start[0]][start[1]] = 0 #(known value)
#clue_perception[start[0]][start[1]] = clue(start, board, dimension)
death_counter = 0

get_neighbors = []
for i in range(-1,2):
  for j in range(-1,2):
    if i==0 and j==0:
      continue
    get_neighbors.append((i,j))


fringe = []
fringe.append(start)
variable_dict = {}
variable_dict[start] = 0

while(-1 in board_perception):
  #update current position
  #print("knowledge base: ")
  #print(knowledge_base)
  #print("var_dict: ")
  #print(variable_dict)
  if len(fringe) > 0:
    current_position = fringe.pop(0)
    remaining_spaces.remove(current_position)
  elif len(remaining_spaces) > 0:
    current_position = remaining_spaces.pop()
  else: 
    break
  #print("current position: " + str(current_position))
  #check if mine 
  if current_position in mines_set:
    death_counter += 1
    variable_dict[current_position] = 1
    print("random death")
    #knowledge_base.append((current_position[0], current_position[1], -1))
    board_perception[current_position[0]][current_position[1]] = 1

  else:
    knowledge_base.append((current_position[0], current_position[1], clue(current_position, board, dimension)))
    variable_dict[current_position] = 0
    board_perception[current_position[0]][current_position[1]] = 0

 #loop through neighbors
  neighbors = get_valid_neighbors(current_position, remaining_spaces)
  #print("neighbors: " + str(neighbors))
  for i in neighbors: 
    if i in variable_dict:
        continue
    #query if neighbor is clear/mine
    query_mine = (i[0], i[1], 1)
    query_clear = (i[0], i[1], 0)

    #if mine, update board_perception, knowledge_base, and remaining_spaces
    if make_inference(knowledge_base, board_perception, clue(current_position, board, dimension), variable_dict, query_mine):
      board_perception[i[0]][i[1]] = 1
      #knowledge_base.append((i[0], i[1], -1))
      variable_dict[i] = 1
      print("Found mine")
      if i in remaining_spaces:
        remaining_spaces.remove(i)
    #if clear, update board_perception, knowledge_base, remaining_spaces, and add to fringe to be explored
    elif make_inference(knowledge_base, board_perception, clue(current_position, board, dimension), variable_dict, query_clear):
      #print("inferred clear: ")
      #print(i)
      variable_dict[i] = 0
      board_perception[i[0]][i[1]] = 0
      if i not in fringe:
        fringe.append(i)
    #print(variable_dict)
  #loop through neighbors of current_position asserting if they are 0(clear) and 1(mine)
  #compare each assertion to current knowledge_base, use proof by contradiction to determine if true
  #update knowledge base
  #move current_position, check if mine, update board_perception, compute new clue
  #continue loop
print("death_counter: " + str(death_counter))
print(board_perception)
#Test Case: 
#query = (1, 0, 1)
#True = there was a contradiction found
#False = no contradiction
#make_inference(knowledge_base, board_perception, query)

Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
Found mine
death_counter: 0
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0.

In [18]:
def get_valid_neighbors(current_node, remaining_spaces):
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    elif (neighbor_x, neighbor_y) in remaining_spaces:
      neighbors.add((neighbor_x, neighbor_y))
  return neighbors

In [137]:
#evaluate an assertion in knowledge_base 
#variables contains new assertion
def evaluate_assertion(assertion, variables, dimension):
  x,y,v = assertion #(x,y) -> location, v -> value of cell( 0 or 1)
  #print("assertion: " + str(assertion))
  #print(variables)
  sum = 0
  for i in get_neighbors:
    #print("sum: " + str(sum))
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]
    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    #add values of known cells
    value = board_perception[neighbor_x][neighbor_y]
    if value != -1:
      sum += value
    #if neighbor is in variables dictionary, add its value
    elif (neighbor_x, neighbor_y) in variables:
      sum += variables[(neighbor_x, neighbor_y)]
    #cannot compute sum of unkown vales so return truw
    else:
      #print("hidden")
      return True
  return sum == v



In [134]:
#will query all neighbors of current position and call evaluate_assertion for each assertion in knowledge_base
def make_inference(knowledge_base, board_perception, value, variable_dict, query):
  qx, qy, qv = query
  #print("query: " + str(query))
  if value == 0 and qv == 0:
    return True
  if (qx,qy) not in variable_dict:
    variable_dict[(qx,qy)] = 1-qv
  #constraint satisfaction problem -> loop through all possible assignments and find one solution
  #since all possible solutions are 0 or 1, use binary
  satisfiable = False
  flag = True
  for assertion in knowledge_base:
    #check each assertion in knowledge base with our contradicted query
    truth_val = evaluate_assertion(assertion, variable_dict, dimension)

    if not truth_val:
      flag = False
      break
    
  if flag:
    satisfiable = True
    variable_dict.pop((qx,qy))
  #return satisfiability of knowledge base
  #print("satisfiable: " + str(satisfiable))
  return not satisfiable
      


  